In [1]:
import random
class Sentiment:
    NEGATIVE="NEGATIVE"
    NEUTRAL="NEUTRAL"
    POSITIVE="POSITIVE"
class Review:
    def __init__(self,text,score):
        self.text=text
        self.score=score
        self.sentiment=self.get_sentiment()
    def get_sentiment(self):
        if (self.score<=2):
            return Sentiment.NEGATIVE
        elif (self.score==3):
            return Sentiment.NEUTRAL
        else:
            return Sentiment.POSITIVE
class ReviewContainer:
    def __init__(self,reviews):
        self.reviews=reviews
        
    def get_text(self):
        return [x.text for x in self.reviews]
    def get_sentiment(self):
        return [x.sentiment for x in self.reviews]
    
    def evenly_distribute(self):
        negative = list(filter(lambda x: x.sentiment == Sentiment.NEGATIVE, self.reviews))
        positive = list(filter(lambda x: x.sentiment == Sentiment.POSITIVE, self.reviews))
        
        positive_shrunk = positive[:len(negative)]
        self.reviews = negative + positive_shrunk
        random.shuffle(self.reviews)

# Load Data

In [2]:

import json
import os
file_name='D:\\Machine Learning\\Scikit\\books_small_10000.json'
reviews=[]
with open(file_name) as f:
    for line in f:
        review=json.loads(line)
        reviews.append(Review(review['reviewText'],review['overall']))
reviews[5].text

'I hoped for Mia to have some peace in this book, but her story is so real and raw.  Broken World was so touching and emotional because you go from Mia\'s trauma to her trying to cope.  I love the way the story displays how there is no "just bouncing back" from being sexually assaulted.  Mia showed us how those demons come for you every day and how sometimes they best you. I was so in the moment with Broken World and hurt with Mia because she was surrounded by people but so alone and I understood her feelings.  I found myself wishing I could give her some of my courage and strength or even just to be there for her.  Thank you Lizzy for putting a great character\'s voice on a strong subject and making it so that other peoples story may be heard through Mia\'s.'

# Prep Data

In [3]:
from sklearn.model_selection import train_test_split
training,test=train_test_split(reviews,test_size=0.33,random_state=42)

train_container = ReviewContainer(training)
test_container = ReviewContainer(test)

In [4]:
train_container.evenly_distribute()

train_x=train_container.get_text() 
train_y=train_container.get_sentiment()

test_container.evenly_distribute()

test_x=test_container.get_text()
test_y=test_container.get_sentiment()

print(train_y.count(Sentiment.POSITIVE))
print(train_y.count(Sentiment.NEGATIVE))

436
436


#### Bag of words vectorization

In [5]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
vectorizer = TfidfVectorizer()

train_x_vectors=vectorizer.fit_transform(train_x)
test_x_vectors=vectorizer.transform(test_x)

print(train_x[0])
print(train_x_vectors[0].toarray())

Such a fun satire on the Christian religion. I really enjoyed the alien race's amusement with the necklace charms and how in the future it became a car instead of a cross. Makes you think.
[[0. 0. 0. ... 0. 0. 0.]]


# Classification

#### Linear SVM

In [6]:
from sklearn import svm
clf_svm=svm.SVC(kernel='linear')
clf_svm.fit(train_x_vectors,train_y)

test_x[0]

clf_svm.predict(test_x_vectors[0])

array(['POSITIVE'], dtype='<U8')

#### Decision tree

In [7]:
from sklearn.tree import DecisionTreeClassifier
clf_dec=DecisionTreeClassifier()
clf_dec.fit(train_x_vectors,train_y)

clf_dec.predict(test_x_vectors[0])

array(['NEGATIVE'], dtype='<U8')

#### Naive Bayes

In [8]:
from sklearn.naive_bayes import GaussianNB
clf_gnb=GaussianNB()
clf_gnb.fit(train_x_vectors.toarray(),train_y)

clf_gnb.predict(test_x_vectors[0].toarray())

array(['POSITIVE'], dtype='<U8')

#### Logistic Regression

In [9]:
from sklearn.linear_model import LogisticRegression
clf_log=LogisticRegression()
clf_log.fit(train_x_vectors,train_y)

clf_log.predict(test_x_vectors[0])

array(['NEGATIVE'], dtype='<U8')

# Evaluation


#### Mean Accuracy

In [18]:
print(clf_svm.score(test_x_vectors, test_y))
print(clf_dec.score(test_x_vectors, test_y))
print(clf_gnb.score(test_x_vectors.toarray(), test_y))
print(clf_log.score(test_x_vectors, test_y))

0.8076923076923077
0.6514423076923077
0.6610576923076923
0.8052884615384616


#### F1 score

In [11]:
from sklearn.metrics import f1_score
print(f1_score(test_y,clf_svm.predict(test_x_vectors), average=None,labels=[Sentiment.POSITIVE,Sentiment.NEGATIVE]))
# print(f1_score(test_y,clf_dec.predict(test_x_vectors), average=None,labels=[Sentiment.POSITIVE,Sentiment.NEGATIVE]))
# print(f1_score(test_y,clf_gnb.predict(test_x_vectors.toarray()), average=None,labels=[Sentiment.POSITIVE,Sentiment.NEGATIVE]))
# print(f1_score(test_y,clf_log.predict(test_x_vectors), average=None,labels=[Sentiment.POSITIVE,Sentiment.NEGATIVE]))

[0.80582524 0.80952381]


In [12]:
test_set=["nice book"," average","brilliant"]
new_test=vectorizer.transform(test_set)
clf_svm.predict(new_test)

array(['POSITIVE', 'POSITIVE', 'POSITIVE'], dtype='<U8')

### Tuning the model ( With Grid Search )

In [16]:
from sklearn.model_selection import GridSearchCV
parameters={'kernel' : ('linear','rbf'),'C':(1,4,8,16,32)}
svc=svm.SVC()
clf=GridSearchCV(svc, parameters, cv=5)
clf.fit(train_x_vectors, train_y)

GridSearchCV(cv=5, estimator=SVC(),
             param_grid={'C': (1, 4, 8, 16, 32), 'kernel': ('linear', 'rbf')})

In [17]:
print(clf.score(test_x_vectors, test_y))

0.8028846153846154


## Saving Model

#### Save Model

In [19]:
import pickle
with open('D:\\Machine Learning\\Scikit\\models\\sentiment_classifier.pkl' , 'wb') as f:
    pickle.dump(clf,f)

#### Load Model

In [21]:
with open('D:\\Machine Learning\\Scikit\\models\\sentiment_classifier.pkl' , 'rb') as f:
    loaded_clf=(pickle.load(f))

In [23]:
print(test_x[0])

loaded_clf.predict(test_x_vectors[0])

This book is very helpful for those who are suffering from porn addiction. It also made me understand the psychology behind it, and realized it is not so much different from other types of addiction. Maybe most people suffering from this type of condition might be ashamed to seek help, so I hope they can get a hold of this book as a first step.


array(['POSITIVE'], dtype='<U8')